# Import Packages

In [1]:
import pandas as pd
import numpy as np
from gingerit.gingerit import GingerIt
from fuzzywuzzy import fuzz
from loguru import logger
from tqdm import tqdm
import re
tqdm.pandas()

C:\Users\prati\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Load Data

In [3]:
mush_df = pd.read_csv('DataStore/compare_datasets/Mush_DataStore_Listing_QC_results_21_02_2023_15_13_54.csv')
yc_df = pd.read_csv('DataStore/compare_datasets/Yellow_Chimes_DataStore_Listing_QC_results__19_02_2023_17_24_01.csv')
mush_df.fillna('NA',inplace = True)
yc_df.fillna('NA',inplace = True)
ground_truth = pd.read_csv('DataStore/compare_datasets/GroundTruth.csv')
special_char = pd.read_csv('Special characters list.csv')
parser = GingerIt()

In [61]:
# mush_df['ScrapedFor'] = np.array(['Mush']*mush_df.shape[0])

# mush_df.columns

# scraped_req_cols = ['ScrapedFor','brand','ASIN','title','description','bullets']
# gt_req_cols = ['Brand','ASIN','Title','Description','Bulletpoints(3)', 'Grade',
#        'Spell check', 'Dimensions details', 'Sentence case','Grade2']

# mush_df = mush_df[req_cols]

In [4]:
ground_truth.columns

Index(['Sr. No.', 'Brand', 'Category', 'Product Name', 'ASIN', 'Product URL',
       'Sales Frequency', 'Title', 'Description', 'Bulletpoints(3)', 'Grade',
       'Spell check', 'Dimensions details', 'Sentence case', 'A+ content',
       'Grade2', 'Main image', 'Additional images', 'Infographics',
       'Lifestyles', 'Video', 'A+ Content3', 'Info/lifestyle Total',
       'Total Images', 'Info present', 'Lifestyle present',
       'Video present or not', 'A+ content4', 'Image quality',
       'Product information', 'Remark', 'Score', 'Grade5', 'Final Grade',
       'Amazon USA', 'Amazon UK', 'Amazon CA', 'Amazon MX', 'Amazon Germany',
       'Amazon France', 'Amazon Italy', 'Amazon Spain', 'Amazon Netherlands',
       'Amazon Belgium', 'Amazon Sweden ', 'Amazon Poland', 'Amazon Australia',
       'Amazon Japan', 'Amazon Singapore', 'Amazon UAE'],
      dtype='object')

# Helper Functions

## Sentence Case

In [5]:
def sentence_case(brand_name,title,brand_present_title):
    first_word = title.strip().split(' ')[0]
    logger.info('First Word {}'.format(first_word))
#     print(brand_name,first_word,brand_present_title)
    logger.info('Brand Present({}) {}'.format(brand_present_title,brand_present_title==1))
    if brand_present_title==1:
        logger.info('brand name({}) = first word({}) {}'.format(brand_name.strip()[0],first_word[0],brand_name.strip()[0]==first_word[0]))
        if brand_name.strip()[0]==first_word[0]:
            res = 1
        else:
            res = 0
    else:
        logger.info('First letter capital({}) {}'.format(first_word[0],first_word[0].isupper()))
        if first_word[0].isupper():
            res = 1
        else:
            res = 0
#     print(res)
    logger.info('Res {}'.format(res))
    return res

In [6]:
def runGinger(txt,my_tool):
    if txt=='NA':
        return [0,['No Description'],txt]
    # logger.info(txt)
    try:
        sentences = [x.strip().lstrip(', ') for x in txt.strip().split('\n')]
        logger.info('sentences are : {}'.format(sentences))
        flg = 1
        corrections = []
        result = []
        for sentence in sentences:
            if len(sentence)>600:
                logger.info('sentence char len is greater than 600')
                order = "[+-]?\d+\.\d+"
                first_str = re.sub(order, '', sentence)
                lines = re.split( r'[?.!]',first_str)
                for i in lines:
                    parse_res = my_tool.parse(i)
                    if len(parse_res['corrections'])>0:
                        # for corr in parse_res['corrections']:
                        # if corr['definition']!='Accept p'
                        flg = 0
                    result.append(parse_res['result'])
                    corrections.append(parse_res['corrections'])
            else:
                parse_res = my_tool.parse(sentence)
                if len(parse_res['corrections'])>0:
                    flg = 0
                result.append(parse_res['result'])
                corrections.append(parse_res['corrections'])
    except:
        return [0,['API Error'],[txt]]
    return [flg,corrections,result]

## Special Character Check

In [7]:
def special_char_check(x):
    if len(set(special_char['chars'])) - len(set(special_char['chars'].tolist())-set(x))>0:
        return 0
    else:
        return 1

## Get complete Title Flag

In [8]:
def get_Title_flag(data):
    ## Brand Name Present
    bn_check = lambda x,y:1 if y.strip().lower().find(x.strip().lower())==0 else 0

    data['title_brand_present'] = data[['brand','title']].progress_apply(lambda x:bn_check(x.brand,x.title),axis = 1)
    ## Sentence Case
    data['title_sentence_case'] = data[['brand','title',"title_brand_present"]].progress_apply(lambda x:sentence_case(x.brand,x.title,x.title_brand_present),axis = 1)
    ## Spell Check
    # data[['title_spellcheck','title_Corrected_text']] = data['title'].progress_apply(lambda x:spellcheck(x,my_tool)).tolist()
    data[['title_spellcheck','title_corrections','title_Corrected_text']] =  pd.DataFrame(data['title'].progress_apply(lambda x: runGinger(x,parser)).tolist())
    # data['title_Corrected_text'] =  data['title'].progress_apply(lambda x: cb.t5_kes_corrector((x)))
    
    # data['title_spellcheck'] = data[['title_Corrected_text','title']].apply(lambda x: samecheck(x.title_Corrected_text,x.title),axis = 1)
    data['final_title_check_flag'] = data[['title_brand_present','title_sentence_case','title_spellcheck']].product(axis = 1)
    # messages['title_spellcheck'] = ['Check to see if Title is grammatically correct']
    return data['final_title_check_flag']

## Get complete Description Flag

In [9]:
def get_Description_flag(data):
    ## Special Character Check
    data['description_special_chr_check'] = data['description'].apply(lambda x:special_char_check(x))
    ## Characters Constrained
    data['description_char_constrained_2000'] = data['description'].apply(lambda x:1 if len(x.strip())<=2000 else 0)
    ## Multiline Check
    def multiline_check(first_str):
        order = "[+-]?\d+\.\d+"
        first_str = re.sub(order, '', first_str)
        lines = re.split( r'[?.!]',first_str)
        if len(lines)>1:
            return 1
        else:
            return 0
    
    data['description_multiline_check'] = data['description'].apply(lambda x:multiline_check(x))

    ## Spell Check 
    # data[['description_spellcheck','description_Corrected_text']] = data['description'].progress_apply(lambda x:spellcheck(x,my_tool)).tolist()
    data[['description_spellcheck','description_corrections','description_Corrected_text']] =  pd.DataFrame(data['description'].progress_apply(lambda x: runGinger(x,parser)).tolist())
    # data['description_Corrected_text'] =  data['description'].progress_apply(lambda x: cb.t5_kes_corrector((x)))
    
    # data['description_spellcheck'] = data[['description_Corrected_text','description']].apply(lambda x: samecheck(x.description_Corrected_text,x.description),axis = 1)

    ## Final Description check Flag
    data['final_description_check_flag'] = data[['description_special_chr_check','description_char_constrained_2000','description_multiline_check','description_spellcheck']].product(axis = 1)

    return data['final_description_check_flag']

## Get complete BulletPoints Flag

In [10]:
def get_BulletPoints_flag(data):
    ## Special Character check
    data['bullets_special_chr_check'] = data['bullets'].apply(lambda x:special_char_check(x))

    ## Number of bullet points check (atleast 3 points)
    data['bullets_number_check'] = data['bullets'].apply(lambda x:1 if x.count('\n')>=2 else 0)

    ## Bullet Points start with capital letter check
    data['bullets_first_capital_check'] = data['bullets'].apply(lambda x: int(''.join([s.strip().lstrip(', ')[0] for s in x.strip().split('\n')]).isupper()) )

    ## Spell Check
    # data[['bullets_spellcheck','bullets_Corrected_text']] = data['bullets'].progress_apply(lambda x:spellcheck(x,my_tool)).tolist()
    data[['bullets_spellcheck','bullets_corrections','bullets_Corrected_text']] =  pd.DataFrame(data['bullets'].progress_apply(lambda x: runGinger(x,parser)).tolist())
    # data['bullets_Corrected_text'] =  data['bullets'].progress_apply(lambda x: cb.t5_kes_corrector((x)))
    # data['bullets_spellcheck'] = data[['bullets_Corrected_text','bullets']].apply(lambda x: samecheck(x.bullets_Corrected_text,x.bullets),axis = 1)

    ## Final Bullet Points check Flag
    data['final_bullet_point_check_flag'] = data[['bullets_special_chr_check','bullets_number_check','bullets_first_capital_check','bullets_spellcheck']].product(axis = 1)

    return data['final_bullet_point_check_flag']

## Get complete Spell Check Flag

In [11]:
def get_sum(lst):
    return sum(lst)
def get_SpellCheck_flag(data):
    data['final_entire_spellcheck'] = data[['title_spellcheck','description_spellcheck','bullets_spellcheck']].apply(lambda x:1 if get_sum([x.title_spellcheck,x.description_spellcheck,x.bullets_spellcheck])==3 else 0,axis = 1)
    return data['final_entire_spellcheck']

## Get complete Dimension Check Flag

In [12]:
def qc_dim(unit,values):
#     metric = {'meter':['m','meter'],
#     'centimeter':['cm','cms','centimeter'],
#     'millimeter':['mm','millimeter'],
#     'inches':['inch','inches']
#     'litre':['l','lit','litre'],
#     'gram':['g','gm','gram'],
#     'kilogram':['kg','kgms','kilogram']}
#     metric = ['meter','centimeter','millimeter','kilometer','inches','foot']
    metric2 = ['m','cms','mm','km','inches','ft']
    metric_change = [1,0.01,0.001,1000,0.0254,0.0348]
    metric_unit= []
    logger.info('unit for qc {}\nvalue for qc {}'.format(unit,values))
    for u in unit:
        umetric_ratio = []
        for m in metric2:
            umetric_ratio.append(fuzz.ratio(u,m))
        metric_unit.append(metric2[np.array(umetric_ratio).argmax()])
    logger.info('metric_unit {}'.format(metric_unit))
    updated_values = [float(v)*metric_change[metric2.index(i)] for i,v in zip(metric_unit,values)]
    logger.info('updated values {}'.format(updated_values))
    if len(set(metric_unit))>1:
        qc_res = [0,updated_values]
    else:
        qc_res = [1,updated_values]
    
    return qc_res

def format_dim(dim):
    logger.info('initial dim {}'.format(dim))
    dim = dim.replace(' ','').lower()
    dim = dim.replace('x',' ')
    units = []
    for unit in re.finditer("[a-z]+",dim):
        units.append(unit.group())
    values = []
    for val in re.finditer("((\d*[.])?\d+)",dim):
        values.append(val.group())
    logger.info('unit is {} and values are {}'.format(units,values))
    return [units,values]

def check_values(value_list):
    logger.info('value_list for check values {}'.format(value_list))
    max_len = [len(l) for l in value_list]
    if len(set(max_len))>1:
        logger.info('The dimensions length do not match!!')
        return 0
    value_list = [sorted(l) for l in value_list]
    logger.info('after sort {}'.format(value_list))
    same_val_list = np.array(value_list).T.tolist()
    logger.info('transform for each dim {}'.format(same_val_list))
    res = 1
    for v in same_val_list:
        ratio_list = [v[i]/v[0] for i in range(len(v))]
        logger.info('ratio list for list {} {}'.format(v,ratio_list))
        res*=np.prod([1 if (i<=1.05 and i>=0.95) else 0 for i in ratio_list])
    logger.info('res {}'.format(res))
    return res

def get_dimensions(a,text):
    logger.info('ASIN is {}'.format(a))
    logger.info(text)
    iters = re.finditer("(((((\d*[.])?\d+) ?[a-zA-Z]+ ?)[xX] ?(((\d*[.])?\d+) ?[a-zA-Z]+ ?)[xX] ?(((\d*[.])?\d+) ?[a-zA-Z]+ ?))|((((\d*[.])?\d+) ?[a-z]+ ?)[xX] ?(((\d*[.])?\d+) ?[a-zA-Z]+ ?)))|(((((\d*[.])?\d+) ?)[xX] ?(((\d*[.])?\d+) ?)[xX] ?(((\d*[.])?\d+) ?[a-zA-Z]+))|((((\d*[.])?\d+) ?)[xX] ?(((\d*[.])?\d+) ?[a-zA-Z]+)))",text)
    matched_strings = []
    for i in iters:
        matched_strings.append(i.group())
    logger.info('matched_strings {} and length is {}'.format(matched_strings,len(matched_strings)==0))
    if len(matched_strings)==0:
        return 0
    same_unit_in_dim = 1
    multi_units_value = []
    for dim in matched_strings:
        units, values = format_dim(dim)
        if len(units)==1:
            units = units*len(values)
        qc_res = qc_dim(units,values)
        same_unit_in_dim*=qc_res[0]
        multi_units_value.append(qc_res[1])
    # dimres = [1,same_unit_in_dim,check_values(multi_units_value)]
    return check_values(multi_units_value)
            
def get_Dimensions_flag(data):
    data['complete_data'] = data['title']+data['description']+data['bullets']#+
    data['ASIN']
    data['final_dimensionality_check'] = data[['ASIN','complete_data']].progress_apply(lambda x: get_dimensions(x.ASIN,x.complete_data),axis = 1)
    # logger.info(data['dimension_check_res'])
    # data[['final_dimensionality_check','same_unit_in_dim','dimensionality_inter_check']] = data['dimension_check_res'].str.split('__%__')
    # data.drop('dimension_check_res',axis = 1,inplace = True)
    return data['final_dimensionality_check']

## Get complete Spell Check Flag

In [13]:
def get_SentenceCase_flag(data):
    data['final_sentence_case_check'] = data['title_sentence_case']

    return data['final_sentence_case_check']

## Get all the flags

In [14]:
def QC_check1(data):
    logger.info('Title Check Started')
    data['final_title_check_flag'] = get_Title_flag(data)
    logger.info('Title Check Completed!!!')

    logger.info('Description Check Started')
    data['final_description_check_flag'] = get_Description_flag(data)
    logger.info('Description Check Completed!!!')

    logger.info('Bullet Points Check Started')
    data['final_bullet_point_check_flag'] = get_BulletPoints_flag(data)
    logger.info('Bullet Points Check Completed!!!')

    data['Grade1'] = data[['final_title_check_flag','final_description_check_flag','final_bullet_point_check_flag']].sum(axis = 1)
    data['Grade1'] = data['Grade1'].apply(lambda x: 'A' if x==3 else('B' if x>0 else 'C'))

    logger.info('Entile Spell Check Started')
    data['final_entire_spellcheck'] = get_SpellCheck_flag(data.copy())
    logger.info('Entile Spell Check Completed!!!')

    logger.info('Dimensionality Check Started')
    # data[['final_dimensionality_check','same_unit_in_dim','dimensionality_inter_check']] = get_Dimensions_flag(data.copy())
    data['final_dimensionality_check']= get_Dimensions_flag(data.copy())
    logger.info('Dimensionality Check Completed!!!')

    logger.info('Sentence Case Check Started')
    data['final_sentence_case_check'] = get_SentenceCase_flag(data.copy())
    logger.info('Sentence Case Check Completed!!!')

    data['Grade2'] = data[['final_entire_spellcheck','final_dimensionality_check','final_sentence_case_check']].sum(axis = 1)
    data['Grade2'] = data['Grade2'].apply(lambda x: 'A' if x==3 else('B' if x==2 else 'C'))
#     tooltips_df = pd.DataFrame(messages)
#     data.style.set_tooltips(tooltips_df)
#     # st.write(messages)
#     tooltips_df.to_csv('DataStore/DataDict.csv',index=False)
    return data

# Result Analysis

In [14]:
# mush_df.shape

# bn_check = lambda x,y:1 if y.strip().lower().find(x.strip().lower())==0 else 0
# mush_df['title_brand_present'] = mush_df[['brand','title']].apply(lambda x:bn_check(x.brand,x.title),axis = 1)

# mush_df['title_sentence_case'] = mush_df[['brand','title',"title_brand_present"]].apply(lambda x:sentence_case(x.brand,x.title,x.title_brand_present),axis = 1)

# mush_df['title_sentence_case'].value_counts()

# mush_df_gt = ground_truth[ground_truth['ASIN'].isin(mush_df['ASIN'])]

# mush_df['final_dimensionality_check_new']= get_Dimensions_flag(mush_df.copy())

# mush_df[['final_dimensionality_check','final_dimensionality_check_new']].value_counts()

# mush_df[mush_df['ASIN']=='B07P12ZDYF']['final_dimensionality_check_new']



(47, 39)

## Yellow Chimes Get Results

In [ ]:
res_df = QC_check1(yc_df)

2023-02-24 10:12:21.618 | INFO     | __main__:QC_check1:2 - Title Check Started
  0%|          | 0/1911 [00:00<?, ?it/s]2023-02-24 10:12:21.692 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:21.693 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:21.693 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:21.694 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:21.694 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:21.694 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:21.695 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:21.695 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:21.696 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:21.698 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:21.698 | INFO     |

2023-02-24 10:12:21.757 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:21.757 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:21.768 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:21.768 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:21.769 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:21.770 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:21.770 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:21.771 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:21.772 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:21.772 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:21.773 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:21.773 | INFO     | __main__:sentence_case:7 - brand name(Y) = first

2023-02-24 10:12:21.839 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:21.839 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:21.851 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:21.851 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:21.853 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:21.854 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:21.854 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:21.855 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:21.855 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:21.855 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:21.855 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:21.855 | INFO     | __main__:sentence_ca

2023-02-24 10:12:21.936 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:21.937 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:21.937 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:21.938 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:21.939 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:21.939 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:21.939 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:21.939 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:21.939 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:21.939 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:21.939 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:21.939 | INFO     | __main__:sentence_case:3 - Firs

2023-02-24 10:12:22.009 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:22.018 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:22.018 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:22.019 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:22.019 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:22.019 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:22.020 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:22.020 | INFO     | __main__:sentence_case:5 - Brand Present(1) True
2023-02-24 10:12:22.020 | INFO     | __main__:sentence_case:7 - brand name(Y) = first word(Y) True
2023-02-24 10:12:22.021 | INFO     | __main__:sentence_case:19 - Res 1
2023-02-24 10:12:22.021 | INFO     | __main__:sentence_case:3 - First Word Yellow
2023-02-24 10:12:22.021 | INFO     | __main__:sentence_case:5 - Brand Pr

...

 19%|█▉        | 360/1911 [38:21<2:12:48,  5.14s/it]2023-02-24 11:27:19.890 | INFO     | __main__:runGinger:7 - sentences are : ['Size - 6.9 cm x 3 cm', 'Gift for her - ideal valentine, birthday, anniversary gift for someone you love. with our packaging box, you do not need to opt-in for any additional gift packaging. The product comes in a beautiful elegant ready-to-gift box A must have piece to add to your jewellery treasure', 'Brand: Yellow Chimes. It is one of the finest brands all across India for fashion jewellery. This product comes in Yellow Chimes branded packaging. Yellow Chimes brings you a new range of fashion jewellery - designer, classy, charming, elegant, appealing, casual as well as party wear, pocket-friendly, superb quality, Quality plating, skin friendly, non-allergic, attractive, mesmerising and the like Yellow Chimes brings you a new range of fashion earrings - designer, classy, charming, elegant, appealing, casual as well as party wear, pocket-friendly, superb qua

 20%|█▉        | 377/1911 [39:53<2:44:46,  6.44s/it]2023-02-24 11:28:52.031 | INFO     | __main__:runGinger:7 - sentences are : ['Silver toned Thread Shaped White Crystal Stoned Long Chain Eye catched Engraved Dangler Earring. Our Product is perfect for valentines Anniversary Birthday gift for girlfriend and Wife.', 'Size - 11*0.9 cm, Closure is Fancy Ear Wire, Material is Silver Plated and Metal is Alloy.', 'Care Instructions:\xa0Wipe with Soft Cloth after every use. Keep away from Water, sprays or perfumes. Try to store your jewellery in a flat box to avoid accidental scratches.', 'Packaging- Your favorite piece of Jewellery will reach you in an Elegant Yellow Chimes box with safety and precautions. You can leave it to us the job of delivering your Jewellery in original condition to you at your door-step.', 'Brand Vision - Yellow Chimes is the one stop solution for all jewellery lovers- which is affordable and can fit for any occasion you pick to shape your own style and personality.

 20%|██        | 389/1911 [40:41<2:08:24,  5.06s/it]2023-02-24 11:29:40.728 | INFO     | __main__:runGinger:7 - sentences are : ['Dimensions: 8.00 centimeters (L) x 2.00 centimeters (W)', 'Color: Blue,silver', 'Can be worn for parties, with dresses and gowns', 'Material: Silver Plated', 'It is advisable to store jewellery in a zip lock pouch (air tight pouch), keep away from water perfume and other chemicals and clean it with dry and soft cloth', 'Blue crystal dangle earring \xa0-a jewellery to treasure always - not only for the woman who wears it, but in th eyes of beholder. it c b put on with any dress style and any occasion with all kind of outfits', 'Beautiful l at one glance - with intricate hi polish creates glamorous reflections and adds luxurious lproduced under quality control; one by one checking; focus on detail processing. top quality/ latest trend de/anti-allergy', 'Gift for women - ideal valentine, r, anniversary gift for someone y love. with our packaging box, y do not n

 21%|██        | 404/1911 [41:42<1:38:49,  3.93s/it]2023-02-24 11:30:41.617 | INFO     | __main__:runGinger:7 - sentences are : ['BEAUTY-A Must have piece to add to your Jewellery Treasure', 'QUALITY-5 Layer Micro Plated. Thickness of plating: 0.1 Micron, where .03 Micron Plating available in Market.', 'SKIN FRIENDLY- Alloy (97% tin and 3% bismuth). Nickel free and Lead free as per International Standards that makes it very skin friendly.', 'SIZE- The Earring dangling part is 3 cm. Width is 1 cm.', 'GIFT FOR HER! - Ideal Valentine, Birthday, Anniversary gift for someone you ❤ LOVE ❤. With our packaging Box, you do not need to opt-in for any additional gift packaging. The product comes in a beautiful Elegant Ready-to-Gift Box.']


 22%|██▏       | 418/1911 [42:27<1:18:52,  3.17s/it]2023-02-24 11:31:25.863 | INFO     | __main__:runGinger:7 - sentences are : ['Green and White Crystal Dangle Eaarrrings -A Jewellery to Treasure always - not Only for the woman who wears it, but in then Eyes of Beholder!', '5 layer micro Platinum\xa0plated. Thickness of plating: 0.1 micron, where.03 micron plating available in market.', 'Beautiful Looks at One Glance- with Intricate High Polish creates Glamorous Reflections and adds Luxurious Looks.', 'Produced under Quality Control; One by One Checking; Focus on Detail Processing. Top Quality/ Latest Trend Design/Anty-Allergy.', 'Gift for women! - Ideal Valentines, Birthday, Anniversary gift for someone you\xa0❤\xa0love\xa0❤. With our packaging box, you do not need to opt-in for any additional gift packaging. the product comes in a beautiful elegant ready-to-gift box.']


In [32]:
res_df

,ASIN,MRP,aplus_images,aplus_text,brand,bullets,description,image_links,price,ratings,...,bullets_Corrected_text,final_bullet_point_check_flag,Grade1,final_entire_spellcheck,final_dimensionality_check,final_sentence_case_check,Grade2,title_corrections,description_corrections,bullets_corrections
0,B06ZY5QDYM,999,https://images-na.ssl-images-amazon.com/images...,NA,Yellow Chimes,BEAUTY-A Must have piece to add to your Jewel...,NA,https://m.media-amazon.com/images/I/31mQiiCG+x...,299,3.9,...,[ BEAUTY-A Must have piece to add to your Jewe...,0,C,0,0,1,C,"[[{'start': 41, 'text': 'Pcs', 'correct': 'PCs...",[No Description],[API Error]
1,B0751CKLPG,899,https://images-na.ssl-images-amazon.com/images...,NA,Yellow Chimes,BEAUTY-A Must have piece to add to your Jewel...,YELLOW CHIMES brings exclusive Fashion Jewelle...,https://m.media-amazon.com/images/I/41ZQf8NulX...,199,3.4,...,[ BEAUTY-A Must have piece to add to your Jewe...,0,B,0,0,1,C,[[]],"[[{'start': 292, 'text': 'contemprary', 'corre...",[API Error]
2,B071GFLYQ6,"2,499",https://images-na.ssl-images-amazon.com/images...,NA,Yellow Chimes,The sweet energy of this Peruvian Opal bracel...,NA,https://m.media-amazon.com/images/I/51bEG9Mqvb...,189,3.9,...,[ The sweet energy of this Peruvian Opal brace...,0,B,0,0,1,C,[[]],[No Description],[API Error]
3,B06ZZ1TRXJ,NA,https://images-na.ssl-images-amazon.com/images...,NA,Yellow Chimes,"Material: PU Leather \n, QUALITY - Top Quali...",NA,https://m.media-amazon.com/images/I/31KnKJDmU2...,NA,3.9,...,"[ Material: PU Leather \n, QUALITY - Top Qual...",0,C,0,0,1,C,"[[{'start': 85, 'text': 'Wear Leather', 'corre...",[No Description],[API Error]
4,B071JK9TPQ,NA,https://images-na.ssl-images-amazon.com/images...,NA,Yellow Chimes,"? Swan, a Symbol of Pure Love ? with designer...",NA,https://m.media-amazon.com/images/I/514FL1d5sK...,NA,3.9,...,"[ ? Swan, a Symbol of Pure Love ? with designe...",0,C,0,0,1,C,"[[{'start': 7, 'text': 'Chimes Crystal', 'corr...",[No Description],[API Error]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906,B09BQLC5GW,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,0,C,0,0,1,C,[No Description],[No Description],[No Description]
1907,B09BQL31KS,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,0,C,0,0,1,C,[No Description],[No Description],[No Description]
1908,B07P8QZM1B,999,NA,NA,NA,A Craftmanship Work with detailed yet precisi...,NA,https://m.media-amazon.com/images/I/4169UIVYSq...,219,3.9,...,[ A Craftmanship Work with detailed yet precis...,0,C,0,0,1,C,[[]],[No Description],[API Error]
1909,B07K6DZML8,"1,345",NA,NA,NA,Elegant and Eye catched Gold plated Kundan St...,NA,https://m.media-amazon.com/images/I/41PKsFB+3t...,219,3.9,...,[ Elegant and Eye catched Gold plated Kundan S...,0,C,0,0,1,C,[[]],[No Description],[API Error]


In [28]:
yc_df_org['final_dimensionality_check'].value_counts()

0    1908
1       3
Name: final_dimensionality_check, dtype: int64